In [1]:
import sagemaker
from sagemaker.local import LocalSession

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
prefix = "sagemaker/Demo-pytorch-mnist"

role = sagemaker.get_execution_role()

In [2]:
role

'arn:aws:iam::499554417458:role/service-role/AmazonSageMaker-ExecutionRole-20220110T125843'

In [3]:
pip install torchvision==0.5.0 --no-cache-dir

     |████████████████████████████████| 4.0 MB 23.5 MB/s            
     |███████████████████████████▏    | 639.6 MB 84.7 MB/s eta 0:00:02 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |██████████████████████████████  | 705.1 MB 88.4 MB/s eta 0:00:01 

     |██████████████████████████████▏ | 709.7 MB 88.4 MB/s eta 0:00:01 

     |██████████████████████████████▍ | 714.2 MB 88.4 MB/s eta 0:00:01 

...

     |███████████████████████████████▌| 740.9 MB 88.4 MB/s eta 0:00:01 

     |███████████████████████████████▊| 746.1 MB 88.4 MB/s eta 0:00:01 

     |████████████████████████████████| 750.9 MB 88.4 MB/s eta 0:00:01 

     |████████████████████████████████| 753.4 MB 88.4 MB/s             


  Attempting uninstall: torch
    Found existing installation: torch 1.7.1
    Uninstalling torch-1.7.1:
      Successfully uninstalled torch-1.7.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.8.2
    Uninstalling torchvision-0.8.2:
      Successfully uninstalled torchvision-0.8.2
Note: you may need to restart the kernel to use updated packages.


In [5]:
from torchvision import datasets, transforms

In [9]:
datasets.MNIST(
    "data",
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(),transforms.Normalize((0.1307,),(0.3081,))]
    ),
)

0it [00:00, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


0it [00:00, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


0it [00:00, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


0it [00:00, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [10]:
inputs = sagemaker_session.upload_data(path="data",bucket=bucket,key_prefix=prefix)
print("input spec (in this case just an S3 path): {}".format(inputs))

input spec (in this case just an S3 path): s3://sagemaker-us-east-1-499554417458/sagemaker/Demo-pytorch-mnist


In [11]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    entry_point = "mnist.py",
    role = role,
    framework_version = "1.4.0",
    py_version = "py3",
    instance_count = 2,
    instance_type = "ml.c4.xlarge",
    hyperparameters = {"epochs":6,"backend":"gloo"}
)

In [12]:
estimator.fit({"training":inputs})

2022-01-10 08:59:53 Starting - Starting the training job...
2022-01-10 09:00:16 Starting - Launching requested ML instancesProfilerReport-1641805192: InProgress
......
2022-01-10 09:01:17 Starting - Preparing the instances for training............
2022-01-10 09:03:21 Downloading - Downloading input data...
2022-01-10 09:03:53 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-01-10 09:04:10,767 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-01-10 09:04:10,770 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2022-01-10 09:04:10,782 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.

2022-01-10 09:04:18 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in

Train Epoch: 1 [6400/30000 (21%)] Loss: 2.076616
Train Epoch: 1 [12800/30000 (43%)] Loss: 1.215920
Train Epoch: 1 [12800/30000 (43%)] Loss: 1.057926
Train Epoch: 1 [19200/30000 (64%)] Loss: 0.914584
Train Epoch: 1 [19200/30000 (64%)] Loss: 0.937029
Train Epoch: 1 [25600/30000 (85%)] Loss: 0.850368
Train Epoch: 1 [25600/30000 (85%)] Loss: 0.673603
Test set: Average loss: 0.3227, Accuracy: 9100/10000 (91%)
Test set: Average loss: 0.3227, Accuracy: 9100/10000 (91%)
Train Epoch: 2 [6400/30000 (21%)] Loss: 0.496909
Train Epoch: 2 [6400/30000 (21%)] Loss: 0.546118
Train Epoch: 2 [12800/30000 (43%)] Loss: 0.378099
Train Epoch: 2 [12800/30000 (43%)] Loss: 0.466322
Train Epoch: 2 [19200/30000 (64%)] Loss: 0.544192
Train Epoch: 2 [19200/30000 (64%)] Loss: 0.678142
Train Epoch: 2 [25600/30000 (85%)] Loss: 0.493608
Train Epoch: 2 [25600/30000 (85%)] Loss: 0.646309
Test set: Average loss: 0.1935, Accuracy: 9417/10000 (94%)
Test set: Average loss: 0.1935, Accuracy: 9417/10000 (94%)
Train Epoch: 3 [6

In [13]:
estimator.model_data

's3://sagemaker-us-east-1-499554417458/pytorch-training-2022-01-10-08-59-52-453/output/model.tar.gz'

In [14]:
predictor = estimator.deploy(initial_instance_count = 1,instance_type="ml.t2.medium")

----------!

In [16]:
!ls data/MNIST/raw

t10k-images-idx3-ubyte	   train-images-idx3-ubyte
t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte	   train-labels-idx1-ubyte
t10k-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


In [18]:
import gzip
import numpy as np
import random
import os

data_dir = "data/MNIST/raw"
with gzip.open(os.path.join(data_dir,"t10k-images-idx3-ubyte.gz"), "rb") as f:
    images = np.frombuffer(f.read(), np.int8, offset=16).reshape(-1,28,28).astype(np.float32)
    
mask = random.sample(range(len(images)), 16)
mask = np.array(mask, dtype=np.int)
data = images[mask]

In [19]:
response = predictor.predict(np.expand_dims(data,axis=1))
print("Raw prediction result")
print(response)


labeled_predictions = list(zip(range(10), response[0]))
print("Labeled predictions:")
print(labeled_predictions)
print()

labeled_predictions.sort(key=lambda label_and_prob: 1.0 - label_and_prob[1])
print("Most likely answer: {}".format(labeled_predictions[0]))

Raw prediction result
[[-4.08968811e+01 -3.26108246e+01 -5.55530167e+01 -1.76888256e+01
  -2.13884010e+01 -1.79258734e-02 -1.79253387e+01 -5.70791206e+01
  -4.03051043e+00 -1.39905796e+01]
 [-4.45184708e+01 -7.86087952e+01  0.00000000e+00 -5.41134758e+01
  -6.46035919e+01 -5.37232475e+01 -3.59026680e+01 -6.33541794e+01
  -2.93396225e+01 -5.96468315e+01]
 [-6.59611664e+01 -1.21969635e+02 -1.06382179e+02 -8.45572052e+01
  -1.13481705e+02  0.00000000e+00 -2.38585968e+01 -1.35927887e+02
  -3.47267075e+01 -1.01389153e+02]
 [-4.74487267e+01 -6.80337067e+01 -4.13311310e+01 -2.86172142e+01
  -7.87157898e+01 -2.33286425e-01 -3.37499161e+01 -7.86159134e+01
  -1.56986475e+00 -5.47533646e+01]
 [-4.74538689e+01 -6.79947586e+01 -3.24823151e+01 -1.03801250e+01
  -5.97275696e+01 -2.46615353e+01 -4.67879372e+01 -6.55552292e+01
  -3.09939351e-05 -2.73011360e+01]
 [-3.05383167e+01 -2.33431358e+01 -1.03711545e-05 -1.15610628e+01
  -3.81892395e+01 -3.13324184e+01 -3.49717941e+01 -1.67495575e+01
  -1.399177

In [20]:
from IPython.display import HTML
HTML(open("input.html").read())

In [26]:
import numpy as np
image = np.array([data],dtype=np.float32)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

0


In [27]:
predictor.delete_endpoint()